In [1]:
from mta_graph import SubwayGraph
from complexes import ComplexesData
import pandas as pd

complexes = ComplexesData()
mta = SubwayGraph()

In [2]:
mta.build_graph()

In [8]:
for line in ["A", "B", "C", "D", "E", "F", "G", "J", "L", "M", "N", "Q", "R", "W", "Z", "1", "2", "3", "4", "5", "6", "7"]:
    if len(mta.ordered_stops(line, 0))!=len(mta.ordered_stops(line, 1)): print(line)

W


In [9]:
def get_onboards_data(complex_id):
    from socrata_od_client import get_ridership_data

    # Get weekday ridership for May 2024 at 9am from station 623
    df = get_ridership_data(
        year=2025,
        month=2,
        day_of_week="Tuesday",
        hour_of_day="9",
        origin_station_complex_id=complex_id
    )

    df_ = df.sort_values('estimated_average_ridership', ascending=False)
    return df_

df_onboards_120 = get_onboards_data("120")
df_onboards_120

,year,month,day_of_week,hour_of_day,origin_station_complex_id,origin_station_complex_name,destination_station_complex_id,destination_station_complex_name,estimated_average_ridership
157,2025,2,Tuesday,9,120,Bedford Av (L),602,"14 St-Union Sq (L,N,Q,R,W,4,5,6)",324.6645
209,2025,2,Tuesday,9,120,Bedford Av (L),610,"Grand Central-42 St (S,4,5,6,7)",192.6793
29,2025,2,Tuesday,9,120,Bedford Av (L),618,"14 St (A,C,E)/8 Av (L)",179.8185
110,2025,2,Tuesday,9,120,Bedford Av (L),611,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",116.7937
212,2025,2,Tuesday,9,120,Bedford Av (L),164,"34 St-Penn Station (A,C,E)",107.6208
...,...,...,...,...,...,...,...,...,...
189,2025,2,Tuesday,9,120,Bedford Av (L),603,"149 St-Grand Concourse (2,4,5)",0.2763
77,2025,2,Tuesday,9,120,Bedford Av (L),340,"Grand Army Plaza (2,3)",0.2763
117,2025,2,Tuesday,9,120,Bedford Av (L),3,"30 Av (N,W)",0.2733
119,2025,2,Tuesday,9,120,Bedford Av (L),71,8 Av (N),0.2733


In [47]:
# 0 = north/east-bound, 1 = south/west-bound
def get_boardings(complex_id: str, line: str, direction: int, df_ : any ,onboarding : bool):
    # Initialize variables
    num_stations = complexes.get_number_of_stations(complex_id) # number of platforms at complex
    boardings = 0
    boardings_df = pd.DataFrame(columns=df_.columns)
    print("Num stations: ", num_stations)

    # If there is only one station, we can just divide the ridership by the number of lines that go through that station
    if num_stations == 1:
        num_lines = mta.lines_at_complex_id(complex_id)
        print("1 station and number of lines: ", num_lines)
        # Get all stops before the current station
        # previous_stops = []
        if onboarding:
            stops_after = get_ordered_stops(line, direction, 1)
            stops_after = stops_after[stops_after.index(int(complex_id)) + 1:]
            print(f"Stops after stop {complex_id}: ", stops_after)
            df_ = df_[df_["destination_station_complex_id"].isin(stops_after)]
            previous_stops = stops_after
        else:
            stops_before = get_ordered_stops(line, direction, 1)
            stops_before = stops_before[:stops_before.index(int(complex_id))]
            df_ = df_[df_["origin_station_complex_id"].isin(stops_before)].copy()
            print("Stops before: ", stops_before)
            previous_stops = stops_before
        
        boardings += sum(df_["estimated_average_ridership"]) / len(num_lines)
        boardings_df = df_ 
        boardings_df["estimated_average_ridership"] = boardings_df["estimated_average_ridership"] / len(num_lines)

    # If there is more than one station
    elif num_stations > 1:
        print("More than 1 station")
        # For each destination station
        for i in range(len(df_)):
            # Get the lines that directly connect the origin and destination station
            connecting_lines = []

            final_complex_id = ""
            if onboarding:
                final_complex_id = str(df_.iloc[i]["destination_station_complex_id"])
            else:
                final_complex_id = str(df_.iloc[i]["origin_station_complex_id"])
            
            connecting_lines = mta.connecting_lines(final_complex_id, complex_id)

            # If there are lines that directly connect the origin and destination station
            if len(connecting_lines) > 0:
                # And if the line we are looking for is one of the connecting lines
                if line in connecting_lines:
                    boardings = boardings + (df_.iloc[i]["estimated_average_ridership"] / len(connecting_lines))

                    df_tmp = df_.iloc[[i]].copy()
                    df_tmp["estimated_average_ridership"] /= len(connecting_lines)
                    boardings_df =  pd.concat([boardings_df, df_tmp], ignore_index=True)
            # If there are no lines that directly connect the origin and destination station
            else:
                # Get all the lines that connect the origin and destination station
                # print("No direct connection: ", df_.iloc[i]["destination_station_complex_name" if onboarding else "origin_station_complex_name"], df_.iloc[i]["estimated_average_ridership"])
                lines = set()
                for path in mta.all_shortest_paths(final_complex_id, complex_id):
                    connecting_lines = mta.connecting_lines(path[0], path[1])
                    lines.update(connecting_lines)

                if line in lines:
                    boardings + df_.iloc[i]["estimated_average_ridership"] / len(lines)

                    df_tmp = df_.iloc[[i]].copy()
                    df_tmp["estimated_average_ridership"] /= len(lines)
                    boardings_df =  pd.concat([boardings_df, df_tmp], ignore_index=True)
    
    boardings_df = boardings_df.sort_values('estimated_average_ridership', ascending=False)
    return boardings, boardings_df

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2188220550.py, line 18)

In [35]:
# 0 = north/east-bound, 1 = south/west-bound
brds, df = get_boardings("120", "L", direction=1, df_=df_onboards_120, onboarding=True)
print("Total offboardings: ", brds)
(df)

Num stations:  1
1 station and num lines:  ['L']
[119, 118, 602, 601, 618]
Total offboardings:  729.973


/var/folders/vv/8v01rf7n71748vxd0m5kt_dr0000gn/T/ipykernel_40804/1239840872.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boardings_df["estimated_average_ridership"] = boardings_df["estimated_average_ridership"] / len(num_lines)


,year,month,day_of_week,hour_of_day,origin_station_complex_id,origin_station_complex_name,destination_station_complex_id,destination_station_complex_name,estimated_average_ridership
157,2025,2,Tuesday,9,120,Bedford Av (L),602,"14 St-Union Sq (L,N,Q,R,W,4,5,6)",324.6645
29,2025,2,Tuesday,9,120,Bedford Av (L),618,"14 St (A,C,E)/8 Av (L)",179.8185
175,2025,2,Tuesday,9,120,Bedford Av (L),601,"14 St (F,M,1,2,3)/6 Av (L)",99.8065
75,2025,2,Tuesday,9,120,Bedford Av (L),119,1 Av (L),72.1755
13,2025,2,Tuesday,9,120,Bedford Av (L),118,3 Av (L),53.5080


In [46]:
complexes.get_names_of_stations("618")

['14 St', '8 Av']

In [10]:
df_onboards_120

,year,month,day_of_week,hour_of_day,origin_station_complex_id,origin_station_complex_name,destination_station_complex_id,destination_station_complex_name,estimated_average_ridership
157,2025,2,Tuesday,9,120,Bedford Av (L),602,"14 St-Union Sq (L,N,Q,R,W,4,5,6)",324.6645
209,2025,2,Tuesday,9,120,Bedford Av (L),610,"Grand Central-42 St (S,4,5,6,7)",192.6793
29,2025,2,Tuesday,9,120,Bedford Av (L),618,"14 St (A,C,E)/8 Av (L)",179.8185
110,2025,2,Tuesday,9,120,Bedford Av (L),611,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",116.7937
212,2025,2,Tuesday,9,120,Bedford Av (L),164,"34 St-Penn Station (A,C,E)",107.6208
...,...,...,...,...,...,...,...,...,...
189,2025,2,Tuesday,9,120,Bedford Av (L),603,"149 St-Grand Concourse (2,4,5)",0.2763
77,2025,2,Tuesday,9,120,Bedford Av (L),340,"Grand Army Plaza (2,3)",0.2763
117,2025,2,Tuesday,9,120,Bedford Av (L),3,"30 Av (N,W)",0.2733
119,2025,2,Tuesday,9,120,Bedford Av (L),71,8 Av (N),0.2733


In [11]:
mta.connecting_lines("120", "164")

[]

In [12]:
complexes.get_names_of_stations("618")

['14 St', '8 Av']

In [13]:
mta.all_shortest_paths("120", "164")

[['120', '618', '164'], ['120', '621', '164']]

In [14]:
for path in mta.all_shortest_paths("120", "164"):
    print(mta.connecting_lines(path[0], path[1]))

['L']
['L']


In [16]:
# 0 = north/east-bound, 1 = south/west-bound
def get_ordered_stops(line : str, direction: int, return_type : int = 0):
    complex_ids = [int(complexes.get_complex_id_by_gtfs_stop_id(stop[:-1])) for stop in mta.ordered_stops(line, direction)]
    complex_names = [complexes.get_station_name_by_gtfs_id(stop[:-1]) for stop in mta.ordered_stops(line, direction)]
    if return_type == 0:
        return dict(zip(complex_names, complex_ids))
    elif return_type == 1:
        return complex_ids
    else:
        return complex_names

In [61]:
list_stops = get_ordered_stops ("L", 0, 1)

stops_before = list_stops[:list_stops.index(int("120"))]
type(stops_before[0])
# stops_before = list_stops[:list_stops.index(138)]
# df_offboards_618[df_offboards_618["origin_station_complex_id"].isin(stops_before)]


int

In [81]:
df_onboards_120[0:5]

,year,month,day_of_week,hour_of_day,origin_station_complex_id,origin_station_complex_name,destination_station_complex_id,destination_station_complex_name,estimated_average_ridership
157,2025,2,Tuesday,9,120,Bedford Av (L),602,"14 St-Union Sq (L,N,Q,R,W,4,5,6)",324.6645
209,2025,2,Tuesday,9,120,Bedford Av (L),610,"Grand Central-42 St (S,4,5,6,7)",192.6793
29,2025,2,Tuesday,9,120,Bedford Av (L),618,"14 St (A,C,E)/8 Av (L)",179.8185
110,2025,2,Tuesday,9,120,Bedford Av (L),611,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",116.7937
212,2025,2,Tuesday,9,120,Bedford Av (L),164,"34 St-Penn Station (A,C,E)",107.6208


In [58]:
df_onboards_120.dtypes

year                                  int64
month                                 int64
day_of_week                          object
hour_of_day                           int64
origin_station_complex_id             int64
origin_station_complex_name          object
destination_station_complex_id        int64
destination_station_complex_name     object
estimated_average_ridership         float64
dtype: object

In [88]:
def get_onboardings(origin_complex_id: str, line: str, direction: int, df_ : any):
    onboardings = 0
    boardings_df = pd.DataFrame(columns=df_.columns)

    for i in range(len(df_)):
        print()
        destination_complex_id = str(df_.iloc[i]["destination_station_complex_id"])
        connecting_lines = mta.connecting_lines(origin_complex_id, destination_complex_id)
        print(f"Destination complex id: {destination_complex_id}")
        print(f"Connecting lines: {connecting_lines}")

        if len(connecting_lines) > 0:
            if line in connecting_lines:
                stops_after = get_ordered_stops(line, direction, 1)
                stops_after = stops_after[stops_after.index(int(origin_complex_id)) + 1:]

                print(f"Stops after stop {origin_complex_id}: {stops_after}")

                if int(destination_complex_id) in stops_after:
                    print(f"{destination_complex_id} is in {stops_after}")
                    onboardings += df_.iloc[i]["estimated_average_ridership"] / len(connecting_lines)
                    df_tmp = df_.iloc[[i]].copy()
                    df_tmp["estimated_average_ridership"] /= len(connecting_lines)
                    boardings_df =  pd.concat([boardings_df, df_tmp], ignore_index=True)
                else:
                    print(f"{destination_complex_id} is not in stops_after")

        else:
            shortest_paths = mta.all_shortest_paths(origin_complex_id, destination_complex_id)
            print(f"From {origin_complex_id} to {destination_complex_id}, shortest paths: {shortest_paths}")

            total_paths = 0
            num_paths = 0
            for path in shortest_paths:
                connections = mta.connecting_lines(path[0], path[1])
                print(f"From {path[0]} to {path[1]}, connections: {connections}")
                total_paths += len(connections)
                if line in connections:
                    if int(path[1]) in stops_after:
                        num_paths += 1
            print(f"Total paths: {total_paths}, num paths: {num_paths}")
            if total_paths > 0 and num_paths > 0:
                onboardings += df_.iloc[i]["estimated_average_ridership"] * (num_paths / total_paths)
                df_tmp = df_.iloc[[i]].copy()
                df_tmp["estimated_average_ridership"] *= (num_paths / total_paths)
                boardings_df =  pd.concat([boardings_df, df_tmp], ignore_index=True)

    return onboardings, boardings_df

In [89]:
obs, df =get_onboardings ("120", "L", 1, df_onboards_120[0:5])


Destination complex id: 602
Connecting lines: ['L']
Stops after stop 120: [629, 122, 123, 124, 125, 126, 127, 630, 129, 130, 131, 621, 133, 134, 135, 136, 137, 138]
602 is not in stops_after

Destination complex id: 610
Connecting lines: []
From 120 to 610, shortest paths: [['120', '602', '610']]
From 120 to 602, connections: ['L']
Total paths: 1, num paths: 0

Destination complex id: 618
Connecting lines: ['L']
Stops after stop 120: [629, 122, 123, 124, 125, 126, 127, 630, 129, 130, 131, 621, 133, 134, 135, 136, 137, 138]
618 is not in stops_after

Destination complex id: 611
Connecting lines: []
From 120 to 611, shortest paths: [['120', '602', '611'], ['120', '601', '611'], ['120', '618', '611'], ['120', '621', '611']]
From 120 to 602, connections: ['L']
From 120 to 601, connections: ['L']
From 120 to 618, connections: ['L']
From 120 to 621, connections: ['L']
Total paths: 4, num paths: 1

Destination complex id: 164
Connecting lines: []
From 120 to 164, shortest paths: [['120', '61

/var/folders/vv/8v01rf7n71748vxd0m5kt_dr0000gn/T/ipykernel_40804/2089473856.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  boardings_df =  pd.concat([boardings_df, df_tmp], ignore_index=True)


In [85]:
print("Onboardings: ", obs)
df

Onboardings:  83.008825


,year,month,day_of_week,hour_of_day,origin_station_complex_id,origin_station_complex_name,destination_station_complex_id,destination_station_complex_name,estimated_average_ridership
0,2025,2,Tuesday,9,120,Bedford Av (L),611,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",29.198425
1,2025,2,Tuesday,9,120,Bedford Av (L),164,"34 St-Penn Station (A,C,E)",53.810400


In [82]:
df_onboards_120[0:5]

,year,month,day_of_week,hour_of_day,origin_station_complex_id,origin_station_complex_name,destination_station_complex_id,destination_station_complex_name,estimated_average_ridership
157,2025,2,Tuesday,9,120,Bedford Av (L),602,"14 St-Union Sq (L,N,Q,R,W,4,5,6)",324.6645
209,2025,2,Tuesday,9,120,Bedford Av (L),610,"Grand Central-42 St (S,4,5,6,7)",192.6793
29,2025,2,Tuesday,9,120,Bedford Av (L),618,"14 St (A,C,E)/8 Av (L)",179.8185
110,2025,2,Tuesday,9,120,Bedford Av (L),611,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",116.7937
212,2025,2,Tuesday,9,120,Bedford Av (L),164,"34 St-Penn Station (A,C,E)",107.6208


In [95]:
mta.lines_at_complex_id("621")

['A', 'C', 'J', 'L', 'Z']